In [177]:
""" 
Program: HW2_ID3.py
Programmed By: Khaled Hossain
Description: Homework - 2 the implementation of the ID3 algorithm
"""


import numpy as np
import pandas as pd
import pprint
import random
eps = np.finfo(float).eps
from numpy import log2 as log

In [178]:
training_data = [
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Java', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, False)
] 


In [179]:
level = []
lang = []
tweets = []
phd = []
hired = []

for stats, data in training_data:
    level.append(list(stats.values())[0])
    lang.append(list(stats.values())[1])
    tweets.append(list(stats.values())[2])
    phd.append((list(stats.values())[3]))
    hired.append(data)

dataSet = {'level' : level, 'lang' : lang, 'tweets': tweets, 'phd' : phd, 'hired' : hired}
df = pd.DataFrame(dataSet, columns = ['level', 'lang', 'tweets', 'phd', 'hired'])
df

,level,lang,tweets,phd,hired
0,Senior,Java,no,no,False
1,Senior,Java,no,yes,False
2,Mid,Python,no,no,True
3,Junior,Python,no,no,True
4,Junior,R,yes,no,True
5,Junior,R,yes,yes,False
6,Mid,R,yes,yes,True
7,Senior,Python,no,no,False
8,Senior,R,yes,no,True
9,Junior,Python,yes,no,True


In [180]:
def find_entropy(df):
    label = df.keys()[-1]   
    entropy = 0                                     # Initialize entropy 
    values = df[label].unique()                     # Gets the unique objects 'Yes', 'No'
    for value in values:
        fraction = df[label].value_counts()[value]/len(df[label])
        entropy += -fraction*np.log2(fraction)
    return entropy


# Calculates entropy of each attribute
def find_entropy_attribute(df,attribute):
    label = df.keys()[-1]   
    target_variables = df[label].unique()  # Outputs all 'Yes' and 'No'
    variables = df[attribute].unique()     # Shows different features (Junior, Mid, Senior)
    entropy_each_feature = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[label] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy_each_feature += -fraction2*entropy           
    return abs(entropy_each_feature)


def info_gain(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]


def get_subtree(df, node,value):
    return df[df[node] == value].reset_index(drop=True)


def buildTree(df, tree=None): 
    label = df.keys()[-1]  
    
    node = info_gain(df) # Get value of max Info Gain
    attValue = np.unique(df[node])
    if tree is None:                    
        tree={}
        tree[node] = {}


    for value in attValue:
        subtable = get_subtree(df,node,value)
        clValue,counts = np.unique(subtable['hired'],return_counts=True)                        
        
        #First we Check the purity of subset if it's pure it stops
        if len(counts)==1:                             
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) 

    return tree

In [181]:
testData = ({"level" : "Junior","lang" : "Java","tweets" : "yes","phd" : "no"})
def classify(query,tree,default = None):
    
    for key in list(query.keys()):             # loop through level, langm tweets and phd
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return default
            result = tree[key][query[key]]
            # we start at the root node and go down the tree comparing the query to the values of the node
            # keep going until I get to a leaf node. 
            if isinstance(result,dict):      
                return classify(query,result)

            else:
                return result

In [182]:
t = buildTree(df)
pprint.pprint(t)

{'level': {'Junior': {'phd': {'no': True, 'yes': False}},
           'Mid': True,
           'Senior': {'tweets': {'no': False, 'yes': True}}}}


In [183]:
# handeling missing data 
sample_data = {"level": "intern", "lang": None, "tweets" : None, "phd": None}
df.append(sample_data, ignore_index=True)
t = buildTree(df)
pprint.pprint(t)


{'level': {'Junior': {'phd': {'no': True, 'yes': False}},
           'Mid': True,
           'Senior': {'tweets': {'no': False, 'yes': True}}}}


In [184]:
# tree = ID3(training_data,training_data,training_data.columns[:-1])
# pprint(tree)
data1 = {"level" : "Junior","lang" : "Java","tweets" : "yes","phd" : "no"} 
data2 = {"level" : "Junior","lang" : "Java","tweets" : "yes","phd" : "yes"} 
classify(data2, t)


False

In [185]:
classify(data1, t)

True

In [186]:
# real data set
df1 = pd.read_csv("Buy_Computer.csv")
df1.drop(['id'], inplace = True, axis = 1)
df1.head()


,age,income,student,credit_rating,Buy_Computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_age,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes


In [187]:
def buildTree(df,tree=None): 
    label = df.keys()[-1]   

    #Get attribute with maximum information gain
    node = find_winner(df)
    
    attValue = np.unique(df[node])
    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
    for value in attValue:
        
        subtree = get_subtree(df,node,value)
        clValue,counts = np.unique(subtree['Buy_Computer'],return_counts=True)                        

        #First we Check the purity of subset if it's pure it stops
        if len(counts)==1:                  
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtree) 
                   
    return tree

In [188]:
tree = buildTree(df1)
pprint.pprint(tree)

{'age': {'middle_age': 'yes',
         'senior': {'credit_rating': {'excellent': 'no', 'fair': 'yes'}},
         'youth': {'student': {'no': 'no', 'yes': 'yes'}}}}


In [189]:
testData1 = {"age": 'senior', "income" : 'low', "student" : 'yes', "credit_rating": 'excellent'}
testData2 = {"age": 'youth', "income" : 'low', "student" : 'yes', "credit_rating": 'fair'}
classify(testData1, tree)


'no'

In [190]:
classify(testData2, tree)

'yes'